In [2]:
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt 

import collections
import hashlib
import numbers
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline

from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.layers import base as base_layer
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import partitioned_variables
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import variables as tf_variables
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import nest
from tensorflow.contrib.rnn.python.ops.core_rnn_cell import _Linear
from matplotlib.colors import LinearSegmentedColormap


In [3]:
_INPUT_WEIGHT_NAME = "W_in"
_RECURRENT_WEIGHT_NAME = "W_rec"
## crossing function
def _calcualte_crossings(x,threshold):
    """input :x : a 2D tensor with batch x n 
    outputs a tensor with the same size as x 
    and values of 0 or 1 depending on comparison between 
    x and threshold""" 
    @tf.custom_gradient
    def crossings(x):
        dtype=x.dtype
        shape=x.get_shape()
        thresholds=tf.constant(threshold,shape=[shape[0].value,shape[1].value],dtype=dtype)
        # if it has one row 
        res=tf.greater_equal(x,thresholds)
        def grad(dy):
            # calculate 1-|x|
            temp=1-tf.abs(x)
            dyres=tf.maximum(temp,0.0)
            return dyres
        return tf.cast(res,dtype=dtype), grad
    z=crossings(x)
    return z 

## calculate input current to the neuron
def _linear_rec_in_current(x,y,output_size):
    """input - x : a 2D tensor with batch x n 
    y is a 2D with size batch x m
    outputs a tensor  with size batch x output_size,
    """ 
    shape_x=x.get_shape()
    shape_y=y.get_shape()
    # 
    scope=vs.get_variable_scope()
    with vs.variable_scope(scope) as outer_scope:
        weight_rec=tf.get_variable(_RECURRENT_WEIGHT_NAME,[shape_x[1],output_size]) # [n x n]
        weight_in=tf.get_variable(_INPUT_WEIGHT_NAME,[shape_y[1],output_size]) # [m x n]
        #w_in_test= lambda:tf.constant(0.0,shape=[shape_y[1],shape_y[2]]) # [n x n]
        #w_rec_test= lambda:tf.constant(0.0,shape=[shape_x[1],shape_x[2]]) # [n x n]
        #weight_rec=tf.Variable(initial_value=w_rec_test,dtype=tf.float32) # [n x n]
        #weight_in=tf.Variable(initial_value=w_in_test,dtype=tf.float32) # [n x n]
        # apply_weights 
        #recurrent
        res_rec_final=tf.matmul(x,weight_rec)
        #input
        res_in_final=tf.matmul(y,weight_in)
        # sum both 
        res=tf.add(res_in_final,res_rec_final)
        return res

## define tuples for the cell 
_LSNNStateTuple = collections.namedtuple("LSNNStateTuple", ("v_mem","spike","t_reset", "I_syn"))
_LSNNOutputTuple = collections.namedtuple("LSNNOutputTuple", ("v_mem","spike","t_reset", "I_syn"))

class LSNNStateTuple(_LSNNStateTuple):
  """Tuple used by LSNN Cells for `state_variables `, and output state.
  Stores five elements: `(v_mem,spike, t_reset, I_syn)`, in that order. Where `v_mem` is the hidden state
  , spike is output, `S_rec` and 'S_in' are spike history, and t_reset refractory history.
  Only used when `state_is_tuple=True`.
  """
  __slots__ = ()

  @property
  def dtype(self):
    (v_mem, spike,t_reset, I_syn) = self
    if v_mem.dtype != spike.dtype:
      raise TypeError("Inconsistent internal state: %s vs %s" %
                      (str(v_mem.dtype), str(spike.dtype)))
    return spike.dtype


class LSNNOutputTuple(_LSNNOutputTuple):
  """Tuple used by SNN Cells for output state.
  Stores six elements: `(v_mem,spike,t_reset,I_syn)`, 
  Only used when `output_is_tuple=True`.
  """
  __slots__ = ()

  @property
  def dtype(self):
    (v_mem,spike,t_reset,I_syn) = self
    if v_mem.dtype != spike.dtype:
      raise TypeError("Inconsistent internal state: %s vs %s" %
                      (str(v_mem.dtype), str(spike.dtype)))
    return spike.dtype


## define LSNNcell 
class LSNNCell(tf.contrib.rnn.RNNCell):
  """LSNN Cell
  Args:
    num_units: 
  """
  def __init__(self,
               num_units,
               tau_m=5.0,
               v_theta=1.0,
               v_reset=0.0,
               R_mem=2.0,
               tau_s=5.0,
               tau_refract=3.0,
               dt=1.0,
               activation=None,
               reuse=None,
               kernel_initializer=None,
               bias_initializer=None,
               state_is_tuple=False,
               output_is_tuple=False):
        
    super(LSNNCell, self).__init__(_reuse=reuse)
    self._num_units = num_units
    self.tau_m=tau_m
    self.v_theta=v_theta
    self.v_reset=v_reset
    self.R_mem=R_mem
    self.tau_s=tau_s
    self.tau_refract=tau_refract
    self.dt=dt
    self._activation = activation or math_ops.tanh
    self._kernel_initializer = kernel_initializer
    self._bias_initializer = bias_initializer
    self._state_is_tuple= state_is_tuple
    self._output_is_tuple= output_is_tuple
    self._calculate_crossing= _calcualte_crossings
    self._linear_rec_in_current = _linear_rec_in_current


  @property
  def state_size(self):
    return (LSNNStateTuple(self._num_units,
                          self._num_units,
                          self._num_units,
                          self._num_units) if self._state_is_tuple else self._num_units)
                          

#  @property
#  def output_size(self):
#    return self._num_units
  @property
  def output_size(self):
    return (LSNNOutputTuple(self._num_units,
                          self._num_units,
                          self._num_units,
                          self._num_units) if self._output_is_tuple else self._num_units)
    

  def call(self, inputs, state):
    """ (LSNN).
    Args:
      inputs: `2-D` tensor with shape `[batch_size x input_size]`.
      state: An `SNNStateTuple` of state tensors, shaped as following 
              `[batch_size x self.state_size]`
              `[batch_size x self.state_size]`
              `[batch_size x self.state_size]`
              `[batch_size x self.state_size]`.
              "v_mem","spike","t_reset", "I_syn"
    Returns:
      A pair containing the new output, and the new state as SNNStateTuple
    """
    if self._state_is_tuple:
        v_mem,spike, t_reset, I_syn = state
    else: 
        logging.warn("%s: Please use float ", self)
    # initialize crossing function 
    #if self._calculate_crossing is None:
    #    self._calculate_crossing = _calcualte_crossings(v_mem,self.v_theta)
        
    #if self._tensor_expand_dim is None:
    #    self._tensor_expand_dim = _tensor_expand_dim(spike,inputs,self._num_units)
    
    
    # compoutation 
    # 1-get spikes
    spike_temp=self._calculate_crossing(v_mem,self.v_theta)
    # hold spike at zero for t_reset>0
    t_subtract= tf.subtract(t_reset,1.0)
    t_update=tf.clip_by_value(t_subtract,0.0,100)
        # get eligible neurons for update 
    eligilible_update=tf.cast(tf.equal(t_update,0.0),tf.float32)
    spike=tf.multiply(eligilible_update,spike_temp)
    
    t_reset_new=tf.add(tf.multiply(spike,self.tau_refract),t_update)
    
    
    v_reseting=tf.multiply(self.v_theta,spike)
    v_update=tf.subtract(v_mem,v_reseting)
        
    
    # calculate new Isyn = W*S 
    I_syn_new=self._linear_rec_in_current(spike,inputs,self._num_units)
    
    ## update membrane potential 
        # update refractory period 
    t_subtract= tf.subtract(t_reset,1.0)
    t_update=tf.clip_by_value(t_subtract,0.0,100)
    t_reset_new=tf.add(tf.multiply(spike,self.tau_refract),t_update)
        # get eligible neurons for update 
    eligilible_update=tf.cast(tf.equal(t_reset_new,0.0),tf.float32)
    
    # calculate factor for updating 
    alpha=tf.exp(tf.negative(tf.divide(self.dt,self.tau_m)))
        # update voltage
    v_mem_new=tf.add(tf.scalar_mul(alpha,v_update),tf.scalar_mul(tf.multiply(1-alpha,self.R_mem),I_syn_new))
    
    ## return variables 
    
    if self._state_is_tuple:
        new_state = LSNNStateTuple( v_mem_new, spike,t_reset_new, I_syn_new )
    if self._output_is_tuple:
        new_output = LSNNOutputTuple(v_mem_new, spike,t_reset_new, I_syn_new ) 
    else:
        new_output = spike
    return new_output, new_state


In [4]:
_OUTPUT_WEIGHT_NAME = "W_out"

## calculate input current to the neuron
def _linear_output(x,output_size):
    """input - x : a 2D tensor with batch x n 
    outputs a tensor  with size batch x output_size,
    """ 
    shape_x=x.get_shape()
    # 
    scope=vs.get_variable_scope()
    with vs.variable_scope(scope) as outer_scope:
        weight_out=tf.get_variable(_OUTPUT_WEIGHT_NAME,[shape_x[1],output_size]) # [n x n]

        # apply_weights 
        res_out_final=tf.matmul(x,weight_out)
        return res_out_final



## define LSNNOutcell 
class LSNNOutCell(tf.contrib.rnn.RNNCell):
  """LSNN Cell
  Args:
    num_units: 
  """
  def __init__(self,
               num_units,
               tau_m=5.0,
               dt=1.0,
               reuse=None,
               kernel_initializer=None,
               bias_initializer=None):
        
    super(LSNNOutCell, self).__init__(_reuse=reuse)
    self._num_units = num_units
    self.tau_m=tau_m
    self.dt=dt
    self._kernel_initializer = kernel_initializer
    self._bias_initializer = bias_initializer
    self._linear_rec_in_current = _linear_output


  @property
  def state_size(self):
    return  self._num_units
                          

#  @property
#  def output_size(self):
#    return self._num_units
  @property
  def output_size(self):
    return  self._num_units
    

  def call(self, inputs, state):
    """ (LSNN).
    Args:
      inputs: `2-D` tensor with shape `[batch_size x input_size]`.
      state: An `SNNStateTuple` of state tensors, shaped as following 
              `[batch_size x self.state_size]`
              `[batch_size x self.state_size]`
              `[batch_size x self.state_size]`
              `[batch_size x self.state_size]`.
              "v_mem","spike","t_reset", "I_syn"
    Returns:
      A pair containing the new output, and the new state as SNNStateTuple
    """
    v_mem = state
 
    # calculate new Isyn = W*S 
    I_syn_new=self._linear_rec_in_current(inputs,self._num_units)
    
    ## update membrane potential 
    
    # calculate factor for updating 
    alpha=tf.exp(tf.negative(tf.divide(self.dt,self.tau_m)))
        # update voltage
    v_mem_new=tf.add(tf.scalar_mul(alpha,v_mem),I_syn_new)
    
    ## return variables
    new_state =  v_mem_new
    new_output = v_mem_new
    
    return new_output, new_state


In [5]:
## calculate spikes based on the threshold crossing for the neurons 
def _calculate_spikes(x,threshold):
    """input - x : a 2D tensor with batch x n ex 10*1
    outputs a tensor with size batch x output_size, where outputsize is twice the size of thresholds_size 
    """ 
    shape_x=x.get_shape()
    #
    logging.warn("%s: Please use float ", shape_x)
    x_aux=tf.random_uniform(shape=[shape_x[0].value,shape_x[1].value],dtype=tf.float32)
    logging.warn("%s: Please use float ", x_aux.get_shape())
    res_out=tf.divide(tf.negative(tf.sign(x_aux-threshold)-1),2)
    
    return res_out


## define LSNNOutcell 
class LSNNSimpleInCell(tf.contrib.rnn.RNNCell):
    def __init__(self,
               num_units=40,
               reuse=None,
               kernel_initializer=None,
               bias_initializer=None):
        super(LSNNSimpleInCell, self).__init__(_reuse=reuse)
        self._num_units = num_units
        self._kernel_initializer = kernel_initializer
        self._bias_initializer = bias_initializer
        self._calculate_spikes = _calculate_spikes
    @property
    def state_size(self):
        return  self._num_units
    @property
    def output_size(self):
        return  self._num_units
    
    def call(self, inputs, state):
        spike_state = state
        # calculate new Isyn = W*S 
        new_spikes=self._calculate_spikes(spike_state,inputs)
        new_state =  new_spikes
        new_output = new_spikes
        return new_output, new_state

In [6]:
# uplading mnist data 

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
# Training Parameters
learning_rate = 0.002
training_steps = 5000
batch_size = 10
display_step = 250
total_batch = int(mnist.train.num_examples / batch_size)
print("Total number of batches:", total_batch)

num_input = 1 # MNIST data input (img shape: 28*28)
timesteps = 28 * 28 # timesteps
num_classes = 10 # MNIST total classes (0-9 digits)
num_input_units=40 # hidden layer num of features
num_hidden_units=200 # hidden layer num of features
num_output_units=10 # output layer - classifier 
def SNN(x):
    Input_Cell = LSNNSimpleInCell(num_units=num_input_units)
    LSNN_cell = LSNNCell(num_units=num_hidden_units,state_is_tuple=True,output_is_tuple=False,tau_s=10.0)
    LSNNOut_cell = LSNNOutCell(num_units=num_output_units)
    encoding_spikes, _ = tf.nn.dynamic_rnn(cell=Input_Cell, dtype=tf.float32, inputs=x)
    spikes, _ = tf.nn.dynamic_rnn(cell=LSNN_cell, dtype=tf.float32, inputs=encoding_spikes)
    output_voltage, _ = tf.nn.dynamic_rnn(cell=LSNNOut_cell, dtype=tf.float32, inputs=spikes)
    return output_voltage
# 
tf.reset_default_graph()
graph=tf.Graph()
with graph.as_default():
    # input to graph
    X=tf.placeholder('float',shape=[batch_size,timesteps,num_input])
    # define out
    output_voltages=SNN(X)
    init=tf.global_variables_initializer()

Total number of batches: 5500


In [ ]:
with tf.Session(graph=graph) as sess:
    sess.run(init)
    batch_x,batch_y=mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    outputs=sess.run(output_voltages, feed_dict={X: batch_x})
    
voltages=outputs[9,:,:]
plt.figure(figsize=[15,10])
ax1=plt.subplot(211)
colors_map=cm.viridis(np.linspace(0,1,voltages.shape[1]))
for t in range(num_output_units):
    ax1.plot(voltages[:,t],color=colors_map[t,:])
    plt.xlim([0,timesteps])

ax2=plt.subplot(212)
ax2.imshow(voltages.transpose())
ax2.set_aspect(20)

#ax3=plt.subplot(313)
#ax3.imshow(np.transpose(batch_x[9]))
#ax3.set_aspect(30)


plt.figure()
plt.imshow(batch_x[9].reshape(28,28))
